In [ ]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

matchData = pd.read_csv("PremierLeagueMatchesUpdated.csv").dropna(subset=["Result"]).drop(columns=["Date", "*Additional Stats", "Attendance"])

# Select features and target and train the model
X = matchData.drop(columns=["homeScore","homeXG","awayScore","awayXG","Result"])
y = matchData["Result"]

# Encode the target feature
resultEncoder = LabelEncoder()
yEncoded = resultEncoder.fit_transform(y)
resultMapping = dict(zip(resultEncoder.classes_, range(len(resultEncoder.classes_))))

# Define qualitative features
qualCols = ["Time", "Home Team", "Away Team", "Referee", "Stadium"]
qualColEncoders = {}

for feature in qualCols:
    newLabelEncoder = LabelEncoder()
    qualColEncoders[feature] = newLabelEncoder
    X[feature] = newLabelEncoder.fit_transform(X[feature])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, yEncoded, test_size=0.2, random_state=42)

# Scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define neural network
model = keras.Sequential([
    keras.layers.Input(shape = (X_train_scaled.shape[1],)),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dense(216, activation="relu"),
    keras.layers.Dense(len(resultMapping), activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=64, validation_data=(X_test_scaled, y_test))

modelLosses = pd.DataFrame(model.history.history)
modelLosses.plot()

# Evaluaing the model
predictions = model.predict(X_test_scaled)
predictionsClassLabels = np.argmax(predictions, axis=1)
print(f"Accuracy: {accuracy_score(y_test, predictionsClassLabels)}")


In [ ]:
# Creating an interactive widget

import ipywidgets as widgets
import warnings
from IPython.display import display, clear_output

warnings.filterwarnings("ignore")

style = {"description_width": "initial"}
layout = widgets.Layout(width="500px")

matchday = widgets.BoundedIntText(style=style, layout=layout, value=1, min=1, max=38, description="Enter the matchday: ")
# date = widgets.(description="Select the date: ")
time = widgets.Dropdown(style=style, layout=layout, options=matchData["Time"].unique().tolist(), description="Select the kickoff time: ")
homeTeam = widgets.Dropdown(style=style, layout=layout, options=matchData["Home Team"].unique().tolist(), description="Select the home team: ")
awayTeam = widgets.Dropdown(style=style, layout=layout, options=matchData["Away Team"].unique().tolist(), description="Select the away team: ")
ref = widgets.Dropdown(style=style, layout=layout, options=matchData["Referee"].unique().tolist(), description="Select the referee: ")
stadium = widgets.Dropdown(style=style, layout=layout, options=matchData["Stadium"].unique().tolist(), description="Select the stadium: ")
prevHomePoints = widgets.BoundedIntText(style=style, layout=layout, value=0, min=0, max=111, description="Enter the number of points the home team had before the match: ")
prevAwayPoints = widgets.BoundedIntText(style=style, layout=layout, value=0, min=0, max=111, description="Enter the number of points the away team had before the match: ")

predictButton = widgets.Button(description="Predict winner")

def predictOutcome(b):
    with output:
        clear_output()

        decodedTime = qualColEncoders["Time"].transform([time.value])[0]
        decodedHomeTeam = qualColEncoders["Home Team"].transform([homeTeam.value])[0]
        decodedAwayTeam = qualColEncoders["Away Team"].transform([awayTeam.value])[0]
        decodedReferee = qualColEncoders["Referee"].transform([ref.value])[0]
        decodedStadium = qualColEncoders["Stadium"].transform([stadium.value])[0]

        input = np.array([matchday.value, decodedTime, decodedHomeTeam, decodedAwayTeam, decodedReferee, decodedStadium, prevHomePoints.value, prevAwayPoints.value])
        input = input.reshape(1,8)
        input = scaler.transform(input)

        predictedProbs = model.predict(input)
        predictedClass = np.argmax(predictedProbs, axis=1)[0]
        originalLabels = list(resultMapping.keys())
        predictedOutcome = originalLabels[predictedClass]

        resultToTeamMapping = {"D": "Draw", "A": awayTeam.value, "H": homeTeam.value}

        print("Result probabilities: ")
        for i, label in enumerate(originalLabels):
            print(f"{resultToTeamMapping[label]}: {predictedProbs[0][i]:.4f}")
        print(f"\nPredicted winner: {resultToTeamMapping[predictedOutcome]}")

predictButton.on_click(predictOutcome)
output = widgets.Output()
display(matchday, time, homeTeam, awayTeam, ref, stadium, prevHomePoints, prevAwayPoints, predictButton, output)